In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


In [2]:
main=pd.read_csv('../data/train.csv')
sub=pd.read_csv('../data/store.csv')
data=pd.merge(main,sub,on='Store',how='left')
data['Date']=pd.to_datetime(data['Date'])
data.sort_values(by=['Store','Date'],inplace=True)
data.set_index('Date',inplace=True)
data.head()

/tmp/ipykernel_1666/1517388885.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  main=pd.read_csv('../data/train.csv')


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,
2013-01-01,1,2,0,0,0,0,a,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-02,1,3,5530,668,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-03,1,4,4327,578,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-04,1,5,4486,619,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-05,1,6,4997,635,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [3]:
def make_lag(data,lags,week):
    for lag in range(1,lags+1):
            if lag==0 and week==0:
                  continue
            data['lag_{}_{}'.format(lag,week)]=data.groupby('Store')['Sales'].shift(lag+7*week)
    return data
temp=pd.DataFrame()
temp=data[['Store','Sales']].copy()
print(temp.columns)
make_lag(temp,3,0)
make_lag(temp,1,1)
temp['rolling_mean_7'] = (
    temp
        .groupby('Store')['Sales']
        .rolling(7, min_periods=1)
        .mean()
        .shift(1)
        .reset_index(level=0, drop=True)
)
temp['rolling_std_7'] = (
    temp
        .groupby('Store')['Sales']
        .rolling(7, min_periods=1)
        .std()
        .shift(1)
        .fillna(0)
        .reset_index(level=0, drop=True)
)
temp[temp['Store']==3].head(10)


Index(['Store', 'Sales'], dtype='object')


,Store,Sales,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
Date,,,,,,,,
2013-01-01,3,0,NaN,NaN,NaN,NaN,4691.857143,2484.687735
2013-01-02,3,6823,0.0,NaN,NaN,NaN,0.000000,0.000000
2013-01-03,3,5902,6823.0,0.0,NaN,NaN,3411.500000,4824.589568
2013-01-04,3,6069,5902.0,6823.0,0.0,NaN,4241.666667,3702.142938
2013-01-05,3,4523,6069.0,5902.0,6823.0,NaN,4698.500000,3157.851854
2013-01-06,3,0,4523.0,6069.0,5902.0,NaN,4663.400000,2735.905938
2013-01-07,3,12247,0.0,4523.0,6069.0,NaN,3886.166667,3100.434642
2013-01-08,3,9800,12247.0,0.0,4523.0,NaN,5080.571429,4242.263345
2013-01-09,3,8001,9800.0,12247.0,0.0,0.0,6480.571429,3888.478488


In [4]:
data.head(1)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,
2013-01-01,1,2,0,0,0,0,a,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [5]:
baseline_features=data.copy()
baseline_features.drop(['Customers','CompetitionDistance',	'CompetitionOpenSinceMonth',
                        	'CompetitionOpenSinceYear',	'Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval'],axis=1,inplace=True)
baseline_features['day']=((data.index - data.index.min())).days
to_category=['Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment']
baseline_features[to_category]=baseline_features[to_category].astype('category')
baseline_features['DayofMonth']=data.index.day
baseline_features.head(10)

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,day,DayofMonth
Date,,,,,,,,,,,
2013-01-01,1,2,0,0,0,a,1,c,a,0,1
2013-01-02,1,3,5530,1,0,0,1,c,a,1,2
2013-01-03,1,4,4327,1,0,0,1,c,a,2,3
2013-01-04,1,5,4486,1,0,0,1,c,a,3,4
2013-01-05,1,6,4997,1,0,0,1,c,a,4,5
2013-01-06,1,7,0,0,0,0,1,c,a,5,6
2013-01-07,1,1,7176,1,1,0,1,c,a,6,7
2013-01-08,1,2,5580,1,1,0,1,c,a,7,8
2013-01-09,1,3,5471,1,1,0,1,c,a,8,9


In [6]:
res=[]
for Store,df_store in data.groupby('Store'):
    fouriers = CalendarFourier(freq="A", order=10)#生成日历傅里叶特征
    dp = DeterministicProcess(
        index=df_store.index,
        constant=True,               #是否包含常数项
        order=1,                     #线性趋势
        seasonal=False,               #是否包含季节性虚拟变量
        additional_terms=[fouriers], #添加傅里叶特征
        drop=True,           
    )
    X=dp.in_sample()  #生成样本内的确定性特征
    X['Store']=Store
    X['Date']=df_store.index
    res.append(X)
X_dp = pd.concat(res, ignore_index=True)
baseline_features=baseline_features.merge(X_dp,on=['Store','Date'],how='left')
y = data["Sales"]               #目标变量
baseline_features.head(10)


/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)


,Store,Date,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,"sin(6,freq=YE-DEC)","cos(6,freq=YE-DEC)","sin(7,freq=YE-DEC)","cos(7,freq=YE-DEC)","sin(8,freq=YE-DEC)","cos(8,freq=YE-DEC)","sin(9,freq=YE-DEC)","cos(9,freq=YE-DEC)","sin(10,freq=YE-DEC)","cos(10,freq=YE-DEC)"
0,1,2013-01-01,2,0,0,0,a,1,c,a,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,1,2013-01-02,3,5530,1,0,0,1,c,a,...,0.103102,0.994671,0.120208,0.992749,0.137279,0.990532,0.154309,0.988023,0.171293,0.985220
2,1,2013-01-03,4,4327,1,0,0,1,c,a,...,0.205104,0.978740,0.238673,0.971100,0.271958,0.962309,0.304921,0.952378,0.337523,0.941317
3,1,2013-01-04,5,4486,1,0,0,1,c,a,...,0.304921,0.952378,0.353676,0.935368,0.401488,0.915864,0.448229,0.893919,0.493776,0.869589
4,1,2013-01-05,6,4997,1,0,0,1,c,a,...,0.401488,0.915864,0.463550,0.886071,0.523416,0.852078,0.580800,0.814046,0.635432,0.772157
5,1,2013-01-06,7,0,0,0,0,1,c,a,...,0.493776,0.869589,0.566702,0.823923,0.635432,0.772157,0.699458,0.714673,0.758306,0.651899
6,1,2013-01-07,1,7176,1,1,0,1,c,a,...,0.580800,0.814046,0.661635,0.749826,0.735417,0.677615,0.801361,0.598181,0.858764,0.512371
7,1,2013-01-08,2,5580,1,1,0,1,c,a,...,0.661635,0.749826,0.746972,0.664855,0.821477,0.570242,0.884068,0.467359,0.933837,0.357698
8,1,2013-01-09,3,5471,1,1,0,1,c,a,...,0.735417,0.677615,0.821477,0.570242,0.891981,0.452072,0.945596,0.325342,0.981306,0.192452
9,1,2013-01-10,4,4892,1,1,0,1,c,a,...,0.801361,0.598181,0.884068,0.467359,0.945596,0.325342,0.984474,0.175531,0.999769,0.021516


In [7]:
baseline_features=baseline_features.merge(right=temp,on=['Date','Store'],how='left')
baseline_features.head()

,Store,Date,DayOfWeek,Sales_x,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,"cos(9,freq=YE-DEC)","sin(10,freq=YE-DEC)","cos(10,freq=YE-DEC)",Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
0,1,2013-01-01,2,0,0,0,a,1,c,a,...,1.000000,0.000000,1.000000,0,NaN,NaN,NaN,NaN,NaN,0.000000
1,1,2013-01-02,3,5530,1,0,0,1,c,a,...,0.988023,0.171293,0.985220,5530,0.0,NaN,NaN,NaN,0.000000,0.000000
2,1,2013-01-03,4,4327,1,0,0,1,c,a,...,0.952378,0.337523,0.941317,4327,5530.0,0.0,NaN,NaN,2765.000000,3910.300500
3,1,2013-01-04,5,4486,1,0,0,1,c,a,...,0.893919,0.493776,0.869589,4486,4327.0,5530.0,0.0,NaN,3285.666667,2908.351137
4,1,2013-01-05,6,4997,1,0,0,1,c,a,...,0.814046,0.635432,0.772157,4997,4486.0,4327.0,5530.0,NaN,3585.750000,2449.327306


In [8]:
baseline_features=baseline_features[baseline_features['Date']>'2013-01-08']
baseline_features[baseline_features['Store']==2].head()

,Store,Date,DayOfWeek,Sales_x,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,"cos(9,freq=YE-DEC)","sin(10,freq=YE-DEC)","cos(10,freq=YE-DEC)",Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
950,2,2013-01-09,3,6763,1,1,0,0,a,a,...,0.325342,0.981306,0.192452,6763,6318.0,6775.0,0.0,0.0,4071.428571,2317.492169
951,2,2013-01-10,4,5618,1,1,0,0,a,a,...,0.175531,0.999769,0.021516,5618,6763.0,6318.0,6775.0,4422.0,4405.857143,2535.198180
952,2,2013-01-11,5,4810,1,1,0,0,a,a,...,0.021516,0.988678,-0.150055,4810,5618.0,6763.0,6318.0,4159.0,4614.285714,2571.239436
953,2,2013-01-12,6,2630,1,0,0,0,a,a,...,-0.133015,0.948362,-0.317191,2630,4810.0,5618.0,6763.0,4484.0,4660.857143,2571.438665
954,2,2013-01-13,7,0,0,0,0,0,a,a,...,-0.284359,0.880012,-0.474951,0,2630.0,4810.0,5618.0,2342.0,4702.000000,2530.125623


In [9]:
baseline_features.head(20)

,Store,Date,DayOfWeek,Sales_x,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,"cos(9,freq=YE-DEC)","sin(10,freq=YE-DEC)","cos(10,freq=YE-DEC)",Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
8,1,2013-01-09,3,5471,1,1,0,1,c,a,...,0.325342,0.981306,0.192452,5471,5580.0,7176.0,0.0,0.0,4585.142857,2231.018633
9,1,2013-01-10,4,4892,1,1,0,1,c,a,...,0.175531,0.999769,0.021516,4892,5471.0,5580.0,7176.0,5530.0,4576.714286,2226.961885
10,1,2013-01-11,5,4881,1,1,0,1,c,a,...,0.021516,0.988678,-0.150055,4881,4892.0,5471.0,5580.0,4327.0,4657.428571,2226.641706
11,1,2013-01-12,6,4952,1,0,0,0,c,a,...,-0.133015,0.948362,-0.317191,4952,4881.0,4892.0,5471.0,4486.0,4713.857143,2226.578349
12,1,2013-01-13,7,0,0,0,0,0,c,a,...,-0.284359,0.880012,-0.474951,0,4952.0,4881.0,4892.0,4997.0,4707.428571,2225.689396
13,1,2013-01-14,1,4717,1,0,0,0,c,a,...,-0.428892,0.785650,-0.618671,4717,0.0,4952.0,4881.0,0.0,4707.428571,2225.689396
14,1,2013-01-15,2,3900,1,0,0,0,c,a,...,-0.563151,0.668064,-0.744104,3900,4717.0,0.0,4952.0,7176.0,4356.142857,1947.844743
15,1,2013-01-16,3,4008,1,0,0,0,c,a,...,-0.683919,0.530730,-0.847541,4008,3900.0,4717.0,0.0,5580.0,4116.142857,1874.016847
16,1,2013-01-17,4,4044,1,0,0,0,c,a,...,-0.788305,0.377708,-0.925925,4044,4008.0,3900.0,4717.0,5471.0,3907.142857,1776.791343
17,1,2013-01-18,5,4127,1,0,0,0,c,a,...,-0.873807,0.213521,-0.976938,4127,4044.0,4008.0,3900.0,4892.0,3786.000000,1726.652928


In [10]:
baseline_features.columns
baseline_features=baseline_features[baseline_features['Open']==1]
baseline_features.sort_values(by='Date',inplace=True)
baseline_features[baseline_features['Store']==78].head(10)

,Store,Date,DayOfWeek,Sales_x,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,"cos(9,freq=YE-DEC)","sin(10,freq=YE-DEC)","cos(10,freq=YE-DEC)",Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
70334,78,2013-01-09,3,3556,1,1,0,0,a,a,...,0.325342,0.981306,0.192452,3556,3712.0,5181.0,0.0,0.0,2932.285714,1552.876869
70335,78,2013-01-10,4,3583,1,1,0,0,a,a,...,0.175531,0.999769,0.021516,3583,3556.0,3712.0,5181.0,3075.0,3001.000000,1570.783244
70336,78,2013-01-11,5,3860,1,1,0,0,a,a,...,0.021516,0.988678,-0.150055,3860,3583.0,3556.0,3712.0,2994.0,3085.142857,1586.047228
70337,78,2013-01-12,6,2797,1,0,0,0,a,a,...,-0.133015,0.948362,-0.317191,2797,3860.0,3583.0,3556.0,3043.0,3201.857143,1612.273077
70339,78,2013-01-14,1,2835,1,0,0,0,a,a,...,-0.428892,0.785650,-0.618671,2835,0.0,2797.0,3860.0,0.0,3241.285714,1596.141568
70340,78,2013-01-15,2,3008,1,0,0,0,a,a,...,-0.563151,0.668064,-0.744104,3008,2835.0,0.0,2797.0,5181.0,2906.142857,1347.982372
70341,78,2013-01-16,3,2727,1,0,0,0,a,a,...,-0.683919,0.530730,-0.847541,2727,3008.0,2835.0,0.0,3712.0,2805.571429,1303.361405
70342,78,2013-01-17,4,2957,1,0,0,0,a,a,...,-0.788305,0.377708,-0.925925,2957,2727.0,3008.0,2835.0,3556.0,2687.142857,1260.777462
70343,78,2013-01-18,5,2976,1,0,0,0,a,a,...,-0.873807,0.213521,-0.976938,2976,2957.0,2727.0,3008.0,3583.0,2597.714286,1207.727855
70344,78,2013-01-19,6,2385,1,0,0,0,a,a,...,-0.938377,0.043022,-0.999074,2385,2976.0,2957.0,2727.0,3860.0,2471.428571,1094.664158


In [11]:
y=pd.DataFrame({'Sales':baseline_features['Sales_x']})
y['log_Sales']=np.log1p(y['Sales'])
baseline_features.drop(['Date','Sales_x','Sales_y'],axis=1,inplace=True)
baseline_features.columns

Index(['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'day', 'DayofMonth', 'const', 'trend',
       'sin(1,freq=YE-DEC)', 'cos(1,freq=YE-DEC)', 'sin(2,freq=YE-DEC)',
       'cos(2,freq=YE-DEC)', 'sin(3,freq=YE-DEC)', 'cos(3,freq=YE-DEC)',
       'sin(4,freq=YE-DEC)', 'cos(4,freq=YE-DEC)', 'sin(5,freq=YE-DEC)',
       'cos(5,freq=YE-DEC)', 'sin(6,freq=YE-DEC)', 'cos(6,freq=YE-DEC)',
       'sin(7,freq=YE-DEC)', 'cos(7,freq=YE-DEC)', 'sin(8,freq=YE-DEC)',
       'cos(8,freq=YE-DEC)', 'sin(9,freq=YE-DEC)', 'cos(9,freq=YE-DEC)',
       'sin(10,freq=YE-DEC)', 'cos(10,freq=YE-DEC)', 'lag_1_0', 'lag_2_0',
       'lag_3_0', 'lag_1_1', 'rolling_mean_7', 'rolling_std_7'],
      dtype='object')

In [12]:
def rmspe(y_true, y_pred):
    """
    计算 RMSPE，但自动忽略 y_true 为 0 的样本。
    防止除以 0 导致的报错。
    """
    # 1. 创建掩码：找出所有真实销量大于 0 的索引
    mask = y_true > 0
    
    # 2. 如果验证集中全是 0 (极少见，但在某些极小的切分下可能发生)，直接返回 0
    if np.sum(mask) == 0:
        return 0.0
    
    # 3. 只取大于 0 的部分进行计算
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    
    # 4. 标准 RMSPE 公式
    score = np.sqrt(np.mean(((y_true_clean - y_pred_clean) / y_true_clean) ** 2))
    
    return score

In [13]:
import re

# 1. 定义清洗函数：将所有非字母数字的字符替换为下划线
def clean_col_name(name):
    # 将 ( ) , = 以及空格替换为 _
    return re.sub(r'[(),=\s-]', '_', str(name))

# 2. 重命名 DataFrame 的所有列
baseline_features.columns = [clean_col_name(c) for c in baseline_features.columns]

# 3. 非常重要：因为列名变了，必须重新生成 feature_cols 列表！
exclude_cols = ['Index', 'Date', 'Sales', 'log_Sales', 'Open']
# 确保排除列也用了清洗过的名字（如果排除列里原本没有特殊字符，其实不需要，但为了保险）
exclude_cols = [clean_col_name(c) for c in exclude_cols]

feature_cols = [c for c in baseline_features.columns if c not in exclude_cols]

print("特征名称已清洗。")
print(f"示例新特征名: {feature_cols}") # 看看清洗后的样子

特征名称已清洗。
示例新特征名: ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'day', 'DayofMonth', 'const', 'trend', 'sin_1_freq_YE_DEC_', 'cos_1_freq_YE_DEC_', 'sin_2_freq_YE_DEC_', 'cos_2_freq_YE_DEC_', 'sin_3_freq_YE_DEC_', 'cos_3_freq_YE_DEC_', 'sin_4_freq_YE_DEC_', 'cos_4_freq_YE_DEC_', 'sin_5_freq_YE_DEC_', 'cos_5_freq_YE_DEC_', 'sin_6_freq_YE_DEC_', 'cos_6_freq_YE_DEC_', 'sin_7_freq_YE_DEC_', 'cos_7_freq_YE_DEC_', 'sin_8_freq_YE_DEC_', 'cos_8_freq_YE_DEC_', 'sin_9_freq_YE_DEC_', 'cos_9_freq_YE_DEC_', 'sin_10_freq_YE_DEC_', 'cos_10_freq_YE_DEC_', 'lag_1_0', 'lag_2_0', 'lag_3_0', 'lag_1_1', 'rolling_mean_7', 'rolling_std_7']


In [14]:
from sklearn.model_selection import TimeSeriesSplit

# 1. 定义 TimeSeriesSplit
# n_splits=5 表示我们将进行5次验证
tscv = TimeSeriesSplit(n_splits=5)
# 1. 定义一组基础参数
# 这里不进行搜索，先用经验参数，目的是快速建立基准
params = {
    'objective': 'regression',       # 回归问题
    'metric': 'rmse',                # 训练时优化 RMSE (对应 log_sales)
    'boosting_type': 'gbdt',
    'verbosity': -1,                 # 不打印乱七八糟的日志
    'n_jobs': -1,                    # 使用所有 CPU 核心
    'seed': 42,
    
    # 以下是一些常见的经验参数，暂不调整
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5
}

# 记录每一折的评分
rmspe_scores = []

print("开始训练基线模型...")

# 2. 开始交叉验证循环
for fold_n, (train_index, val_index) in enumerate(tscv.split(baseline_features)):
    
    # X
    X_train = baseline_features.iloc[train_index]
    X_val   = baseline_features.iloc[val_index]
    
    # y（避免链式索引）
    y_train = y['log_Sales'].iloc[train_index]
    y_val   = y['log_Sales'].iloc[val_index]

    # 真实值用于 RMSPE
    y_true_val = y['Sales'].iloc[val_index].values

    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature='auto')
    val_data   = lgb.Dataset(X_val, label=y_val, categorical_feature='auto', reference=train_data)

    bst = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data, val_data],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=0),
        ]
    )

    preds_log = bst.predict(X_val, num_iteration=bst.best_iteration)
    preds_real = np.expm1(preds_log)

    score = rmspe(y_true_val, preds_real)
    rmspe_scores.append(score)

    print(f"Fold {fold_n+1} RMSPE: {score:.5f}")

# 3. 输出平均分数
print(f"\n基线模型平均 RMSPE: {np.mean(rmspe_scores):.5f}")
print(f"基线模型标准差: {np.std(rmspe_scores):.5f}")

开始训练基线模型...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[82]	training's rmse: 0.166269	valid_1's rmse: 0.18083
Fold 1 RMSPE: 0.19091
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[51]	training's rmse: 0.166103	valid_1's rmse: 0.211204
Fold 2 RMSPE: 0.23723
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[150]	training's rmse: 0.14855	valid_1's rmse: 0.194298
Fold 3 RMSPE: 0.17662
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[418]	training's rmse: 0.131627	valid_1's rmse: 0.167588
Fold 4 RMSPE: 0.31806
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[537]	training's rmse: 0.128581	valid_1's rmse: 0.148853
Fold 5 RMSPE: 0.15012

基线模型平均 RMSPE: 0.21459
基线模型标准差: 0.05896


In [16]:
import optuna
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

# 确保使用的是清洗后(Open=1)的 df_train
target_df = baseline_features.copy()

def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        
        # 针对纯销量的搜索空间
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.15), # 略微调低下限
        'num_leaves': trial.suggest_int('num_leaves', 30, 150),
        'max_depth': trial.suggest_int('max_depth', 8, 15), # 允许更深
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 80),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0), # 稍微放松正则
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
    }

    tscv = TimeSeriesSplit(n_splits=5) # 恢复为5折，求稳
    rmspe_scores = []

    for train_index, val_index in tscv.split(target_df):
        X_train = target_df.iloc[train_index][feature_cols]
        y_train = y.iloc[train_index]['log_Sales']
        X_val = target_df.iloc[val_index][feature_cols]
        y_val = y.iloc[val_index]['log_Sales']
        y_true_val = y.iloc[val_index]['Sales'].values # 真实销量

        train_data = lgb.Dataset(X_train, label=y_train, categorical_feature='auto')
        val_data = lgb.Dataset(X_val, label=y_val, categorical_feature='auto', reference=train_data)

        bst = lgb.train(
            param,
            train_data,
            num_boost_round=1500, # 增加轮数
            valid_sets=[val_data],
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
        )

        preds_log = bst.predict(X_val, num_iteration=bst.best_iteration)
        preds_real = np.expm1(preds_log)
        
        # 因为数据已经清洗过，不需要 mask 也能算，但为了保险还是用 rmspe 函数
        score = rmspe(y_true_val, preds_real)
        rmspe_scores.append(score)

    return np.mean(rmspe_scores)

print("开始最终基线超参搜索...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30) # 建议跑 30 次

print("\n最佳参数:", study.best_params)
print("最佳 CV RMSPE:", study.best_value)

[I 2025-12-11 16:51:17,083] A new study created in memory with name: no-name-b84155c1-c313-4915-8cfa-e33d68f5429c


开始最终基线超参搜索...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[453]	valid_0's rmse: 0.172957
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[53]	valid_0's rmse: 0.207227
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[138]	valid_0's rmse: 0.185677
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[388]	valid_0's rmse: 0.163016
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[558]	valid_0's rmse: 0.1408


[I 2025-12-11 16:54:57,951] Trial 0 finished with value: 0.2048461166020381 and parameters: {'learning_rate': 0.0613264186322728, 'num_leaves': 96, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.9750047731372056, 'colsample_bytree': 0.8310865961477094, 'reg_alpha': 1.972034701244055, 'reg_lambda': 1.1751553096302922}. Best is trial 0 with value: 0.2048461166020381.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[185]	valid_0's rmse: 0.175145
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[35]	valid_0's rmse: 0.206059
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[175]	valid_0's rmse: 0.185865
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[245]	valid_0's rmse: 0.165633
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[549]	valid_0's rmse: 0.141147


[I 2025-12-11 16:56:50,484] Trial 1 finished with value: 0.20700773744140966 and parameters: {'learning_rate': 0.11430238065554159, 'num_leaves': 58, 'max_depth': 12, 'min_child_samples': 56, 'subsample': 0.730505107507882, 'colsample_bytree': 0.9670055753637224, 'reg_alpha': 1.2159640303607662, 'reg_lambda': 1.8415865446595128}. Best is trial 0 with value: 0.2048461166020381.


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[701]	valid_0's rmse: 0.165226
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[54]	valid_0's rmse: 0.206411
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[141]	valid_0's rmse: 0.188282
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[336]	valid_0's rmse: 0.164669
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[602]	valid_0's rmse: 0.138537


[I 2025-12-11 17:52:16,450] Trial 28 finished with value: 0.20032318619205194 and parameters: {'learning_rate': 0.05801820833937896, 'num_leaves': 144, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.9129847936149911, 'colsample_bytree': 0.9384389823955829, 'reg_alpha': 0.6588925840200691, 'reg_lambda': 1.5944281549844126}. Best is trial 28 with value: 0.20032318619205194.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[124]	valid_0's rmse: 0.17013
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[63]	valid_0's rmse: 0.208569
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[182]	valid_0's rmse: 0.188974
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[414]	valid_0's rmse: 0.168149
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[667]	valid_0's rmse: 0.139826


[I 2025-12-11 17:52:54,138] Trial 29 finished with value: 0.2052383912488529 and parameters: {'learning_rate': 0.0586975841518365, 'num_leaves': 99, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.9623130438578477, 'colsample_bytree': 0.9374318606336325, 'reg_alpha': 0.010821091202967037, 'reg_lambda': 1.538848283851871}. Best is trial 28 with value: 0.20032318619205194.



最佳参数: {'learning_rate': 0.05801820833937896, 'num_leaves': 144, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.9129847936149911, 'colsample_bytree': 0.9384389823955829, 'reg_alpha': 0.6588925840200691, 'reg_lambda': 1.5944281549844126}
最佳 CV RMSPE: 0.20032318619205194
